In [2]:
#!/usr/bin/env python3
import os
import re
import numpy as np
import pandas as pd
import rasterio
from rasterio.warp import (
    calculate_default_transform,
    reproject,
    Resampling,
    transform as warp_transform,
    transform_geom,
)
from rasterio.crs import CRS
from rasterio.features import rasterize
from shapely.geometry import mapping, LineString
from multiprocessing import Pool, cpu_count, set_start_method
from tqdm import tqdm
import heapq

# READ-ME


In [ ]:
'''
THIS FILE SHOULD CREATE A DATAFRAME WHICH WILL CONTAIN THE DAYS IN WHICH THE ARCTIC IS NAVIGABLE FOR EACH ROUTE

DEPENDENCIES: numpy, pandas, rasterio, shapely, matplotlib, scikit-image, tqdm, multiprocessing

*** MAKE SURE TO SET THE ICE_THRESHOLD TO THE THREHSOLD YOU ARE CHECKING FOR ***

THERE IS A LINE TO OUTPUT IN CSV FORMAT WHICH IS WHAT IS USED IN OTHER FILES (SPECIFICALLY FOR CREATING THE GRPAHS AND FINAL ANALYSIS)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

KEY FEATURES:
- USES A* PATHFINDING TO COMPUTE THE SHORTEST NAVIGABLE PATH FOR EACH ORIGIN-DESTINATION PAIR
- CONSIDERS ICE CONCENTRATION AND LAND OBSTICLES (LAND PIXELS AND THINK ICE)
- REPROJECTS ICE FORCASTED DATA TO A COMMON BASE MAP (EPSG:3413) WHICH THE A* ALGORITHM USES
- CARVES OUT THE PANAMA AND SUEZ CANALS FROM THE ICE DATA TO ALLOW FOR NAVIGATION THROUGH THESE AREAS TO MATCH REAL-WORLD NAVIGATION

THE NAVIAGBLE DAYS ALGORITHM WORKS AS FOLLOWS:
- IT GOES FORWARD AND BACKWARDS IN TIME FROM SEPTEMBER 1ST TO FIND NAVIGABLE DAYS (SEPTEMBER 1ST IS ARBITRARILY IN THE MIDDLE OF THE ARCTIC NAVIGATION SEASON)
    - THIS WAS DONE TO SPEED UP COMPUTATION TIME AS THE A* ALGORITHM IS COMPUTATIONALLY EXPENSIVE WHEN RUN 4 TIMES PER DAY PER EACH DAY FOR 25 YEARS
- IT GOES FORWARDS AND BACKWARDS IN TIME FOR EACH YEAR IN 1 DAY STEPS
- FOR EACH DAY, IT RUNS THE A* ALGORITHM FOR EACH ORIGIN-DESTINATION PAIR
- IF THE A* ALGORITHM RETURNS A PATH SHORTER THAN THE DISTANCE THRESHOLD IT IS CONSIDERED ARCTIC NAVIGABLE
- THIS REPEATS FORWARDS UNTIL 7 DAYS IN A ROW ARE NOT NAVIGABLE (STOPPING CONDITION)
- THIS REPEATS BACKWARDS UNTIL 7 DAYS IN A ROW ARE NAVIGABLE (STOPPING CONDITION)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

FOR QUESTION OR ISSUES CONTACT: 
- Ioannis Thomopoulos       -->  ioannis.thomopoulos@studbocconi.it
- Jacopo D'Angelo           -->  jacopo.dangelo@studbocconi.it
- Max Rienth                -->  maximilian.rienth@studbocconi.it
- Luca Milani               -->  luca.milani2@studbocconi.it
'''

# SET ICE THRESHOLD


In [ ]:
ICE_THRESHOLD = 150             #CHANGE THIS TO GET OTHER THRESHOLDS ----> {150, 225, 300, 375, 450} 

# PATHS + CONSTANTS


In [4]:
BASE_PATH      = 'created_data/northern_hemisphere_ne.tif'
INPUT_FOLDER   = 'created_data/future_ice_maps'
CRS_LONLAT     = CRS.from_epsg(4326)
CRS_OUT        = CRS.from_epsg(3411)

START_COORDS = [
    (31.284513, 121.985618),  # Shanghai
    (1.193021, 103.669947),   # Singapore
]
GOAL_COORDS  = [
    (40.505985, -73.976818),  # New York
    (52.003414,   4.011638),  # Rotterdam
]
ROUTE_NAMES = [
    'Shanghai to New York',
    'Shanghai to Rotterdam',
    'Singapore to New York',
    'Singapore to Rotterdam',
]

# per-route stop-thresholds (pixels)
ROUTE_THRESHOLDS = {
    'Shanghai to New York':      750,
    'Shanghai to Rotterdam':     750,
    'Singapore to New York':     900,
    'Singapore to Rotterdam':    750,
}

ICE_MAX       = 1000
LAND_VAL      = 3000
NEIGHBORS     = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]

# LOAD BASE FILES + CARVE SUEZ & PANAMA CANAL


In [ ]:
with rasterio.open(BASE_PATH) as src:
    BASE   = src.read(1).astype(np.float32)
    BT     = src.transform
    BCRS   = CRS_LONLAT
    BNDS   = src.bounds
    SW, SH = src.width, src.height

SUEZ_PROJ = transform_geom(
    "EPSG:4326", CRS_OUT.to_string(),
    mapping(LineString([
        (32.302856, 31.371905),
        (32.349123, 30.535917),
        (32.541629, 29.881941),
    ]))
)
PANAMA_PROJ = transform_geom(
    "EPSG:4326", CRS_OUT.to_string(),
    mapping(LineString([
        (-79.926716,  9.377336),
        (-79.730600,  9.119844),
        (-79.545007,  8.911799),
    ]))
)

# UTILITY FUNCTIONS


In [ ]:
def extract_date(fn):
    m = re.search(r'(\d{8})', fn)
    return m.group(1) if m else None

def heuristic(a, b):
    return np.hypot(a[0] - b[0], a[1] - b[1])

def astar(grid, start, goal):
    open_set = [(heuristic(start, goal), start)]
    came_from = {}
    gscore = {start: 0}
    visited = set()
    while open_set:
        _, cur = heapq.heappop(open_set)
        if cur == goal:
            path = [cur]
            while cur in came_from:
                cur = came_from[cur]
                path.append(cur)
            return path[::-1]
        if cur in visited:
            continue
        visited.add(cur)
        for dr, dc in NEIGHBORS:
            nr, nc = cur[0] + dr, cur[1] + dc
            if not (0 <= nr < grid.shape[0] and 0 <= nc < grid.shape[1]):
                continue
            val = grid[nr, nc]
            if np.isnan(val) or val == LAND_VAL or val > ICE_THRESHOLD:
                continue
            cost = np.hypot(dr, dc) * (1 + val / ICE_MAX)
            g = gscore[cur] + cost
            nbr = (nr, nc)
            if g < gscore.get(nbr, np.inf):
                came_from[nbr] = cur
                gscore[nbr] = g
                heapq.heappush(open_set, (g + heuristic(nbr, goal), nbr))
    return None

# PRE PROCESS EACH FILE


In [ ]:
def process_day(tif):
    date = extract_date(tif)
    # 1) Warp ice to base grid
    ice_on_base = np.zeros_like(BASE)
    with rasterio.open(os.path.join(INPUT_FOLDER, tif)) as src_ice:
        reproject(
            source=rasterio.band(src_ice, 1),
            destination=ice_on_base,
            src_transform=src_ice.transform,
            src_crs=src_ice.crs,
            dst_transform=BT,
            dst_crs=BCRS,
            resampling=Resampling.nearest
        )
        ice_res = (abs(src_ice.transform.a), abs(src_ice.transform.e))

    # 2) Merge water/land/ice
    merged_ll = np.where(
        (ice_on_base > 0) & (ice_on_base <= ICE_MAX),
        ice_on_base,
        np.where(BASE == 1, LAND_VAL, 0)
    )

    # 3) Reproject to EPSG:3411
    t_out, w_out, h_out = calculate_default_transform(
        BCRS, CRS_OUT, SW, SH, *BNDS, resolution=ice_res
    )
    merged_out = np.zeros((h_out, w_out), dtype=np.float32)
    reproject(
        source=merged_ll,
        destination=merged_out,
        src_transform=BT,
        src_crs=BCRS,
        dst_transform=t_out,
        dst_crs=CRS_OUT,
        resampling=Resampling.nearest
    )

    # 4) Carve canals
    mask = rasterize(
        [(SUEZ_PROJ, 1), (PANAMA_PROJ, 1)],
        out_shape=merged_out.shape,
        transform=t_out,
        all_touched=True,
        dtype='uint8'
    )
    merged_out[mask == 1] = 0

    # 5) Compute pixel indices of starts/goals
    xs_s, ys_s = warp_transform(
        "EPSG:4326", CRS_OUT,
        *zip(*[(lon, lat) for lat, lon in START_COORDS])
    )
    xs_g, ys_g = warp_transform(
        "EPSG:4326", CRS_OUT,
        *zip(*[(lon, lat) for lat, lon in GOAL_COORDS])
    )
    with rasterio.io.MemoryFile() as mem:
        with mem.open(
            driver='GTiff', dtype='float32', count=1,
            crs=CRS_OUT, transform=t_out,
            width=w_out, height=h_out
        ) as tmp:
            tmp.write(merged_out, 1)
            starts = [tmp.index(x, y) for x, y in zip(xs_s, ys_s)]
            goals  = [tmp.index(x, y) for x, y in zip(xs_g, ys_g)]

    # 6) Run A* for each route and record lengths
    row = {'date': date}
    pairs = [
        (starts[0], goals[0]),  # Shanghai → New York
        (starts[0], goals[1]),  # Shanghai → Rotterdam
        (starts[1], goals[0]),  # Singapore → New York
        (starts[1], goals[1]),  # Singapore → Rotterdam
    ]
    for idx, (s, g) in enumerate(pairs):
        path = astar(merged_out, s, g)
        row[ROUTE_NAMES[idx]] = len(path) if path else np.nan

    return row

# FORWARD + BACKWARD PASSES (THIS IS MAIN CODE)


In [ ]:
if __name__ == '__main__':
    set_start_method('fork', force=True)

    # gather all tif files & their dates
    tif_files = sorted(f for f in os.listdir(INPUT_FOLDER) if f.endswith('.tif'))
    dated     = [(fn, extract_date(fn)) for fn in tif_files if extract_date(fn)]

    processes = min(12, cpu_count())
    results   = []

    for year in range(2025, 2051):
        ystr = str(year)
        year_files = [fn for fn, dt in dated if dt.startswith(ystr)]
        if not year_files:
            continue
        year_files.sort(key=lambda fn: extract_date(fn))

        sept1 = ystr + '0901'
        forward_files  = [f for f in year_files if extract_date(f) >= sept1]
        backward_files = [f for f in year_files if extract_date(f) <  sept1][::-1]

        # ─── Forward pass ───────────────────────────────────────────
        consec = 0
        pool = Pool(processes=processes)
        for row in tqdm(
            pool.imap_unordered(process_day, forward_files, chunksize=1),
            total=len(forward_files),
            desc=f'{year} forward',
            unit='day'
        ):
            if all(row[name] > ROUTE_THRESHOLDS[name] for name in ROUTE_NAMES):
                consec += 1
            else:
                consec = 0
            results.append(row)
            if consec >= 7:
                break
        pool.terminate()
        pool.join()

        # ─── Backward pass ──────────────────────────────────────────
        consec = 0
        pool = Pool(processes=processes)
        for row in tqdm(
            pool.imap_unordered(process_day, backward_files, chunksize=1),
            total=len(backward_files),
            desc=f'{year} backward',
            unit='day'
        ):
            if all(row[name] > ROUTE_THRESHOLDS[name] for name in ROUTE_NAMES):
                consec += 1
            else:
                consec = 0
            results.append(row)
            if consec >= 7:
                break
        pool.terminate()
        pool.join()

    # Build and save single CSV
    all_routes = pd.DataFrame(results)
    all_routes.sort_values('date', inplace=True)
    all_routes.reset_index(drop=True, inplace=True)

# WRITE TO CSV


In [ ]:
all_routes.to_csv(f'created_data/Threshold_{ICE_THRESHOLD}.csv', index=False)              